# Hugginface and Transformers
- Transformer sind keine Actionfiguren sondern eine Deeplearning Architektur 
- Hugginface ist eine open source firma die bestimmte fertige deep learning modelle zur Verfügung stellt
- Wir müssen dazu deeplearning bibliotheken lokal installieren
- Es gibt zwei wichtige: pytorch (Facebook) und tensorflow (Google)
- more https://huggingface.co/transformers/task_summary.html

In [3]:
#!pip install transformers

In [4]:
#!pip install torch torchvision

## Sentiment detection

In [5]:
from transformers import pipeline

# Allocate a pipeline for sentiment-analysis
classifier = pipeline('sentiment-analysis')
classifier('We are very happy to include pipeline into the transformers repository.')

[{'label': 'POSITIVE', 'score': 0.9978193640708923}]

In [6]:
classifier('I am really sad that things had to be so terrible with this lockdown.')

[{'label': 'NEGATIVE', 'score': 0.9989656209945679}]

In [8]:
classifier('This is a neutral sencence. ')

[{'label': 'NEGATIVE', 'score': 0.9598389863967896}]

## Question Answering - ziemlich cool!
sog. Extractive Question Answering

In [9]:
from transformers import pipeline

# Allocate a pipeline for question-answering
question_answerer = pipeline('question-answering')
question_answerer({'question': 'What is the name of the repository ?',
                   'context': 'Pipeline have been included in the huggingface/transformers repository'
                  })

{'score': 0.513595461845398,
 'start': 35,
 'end': 59,
 'answer': 'huggingface/transformers'}

In [12]:
question_answerer({'question': 'How cold is the sun?',
                   'context': 'The Eiffel tower has been built in the 18th century and stands in Paris. '
                  })

{'score': 0.50897616147995, 'start': 66, 'end': 72, 'answer': 'Paris.'}

# Lückentext ausfüllen
sog. Masked Language Modeling

In [13]:
from transformers import pipeline
nlp = pipeline("fill-mask")

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from pprint import pprint
pprint(nlp(f"HuggingFace is creating a {nlp.tokenizer.mask_token} that the community uses to solve NLP tasks."))

[{'score': 0.17927402257919312,
  'sequence': '<s>HuggingFace is creating a tool that the community uses to '
              'solve NLP tasks.</s>',
  'token': 3944,
  'token_str': 'Ġtool'},
 {'score': 0.11349397897720337,
  'sequence': '<s>HuggingFace is creating a framework that the community uses '
              'to solve NLP tasks.</s>',
  'token': 7208,
  'token_str': 'Ġframework'},
 {'score': 0.052435580641031265,
  'sequence': '<s>HuggingFace is creating a library that the community uses to '
              'solve NLP tasks.</s>',
  'token': 5560,
  'token_str': 'Ġlibrary'},
 {'score': 0.034935325384140015,
  'sequence': '<s>HuggingFace is creating a database that the community uses '
              'to solve NLP tasks.</s>',
  'token': 8503,
  'token_str': 'Ġdatabase'},
 {'score': 0.028602493926882744,
  'sequence': '<s>HuggingFace is creating a prototype that the community uses '
              'to solve NLP tasks.</s>',
  'token': 17715,
  'token_str': 'Ġprototype'}]


In [16]:
pprint(nlp(f"The man went skiing with his {nlp.tokenizer.mask_token} not knowing that he would have an accident."))

[{'score': 0.15306979417800903,
  'sequence': '<s>The man went skiing with his girlfriend not knowing that he '
              'would have an accident.</s>',
  'token': 6096,
  'token_str': 'Ġgirlfriend'},
 {'score': 0.07086747139692307,
  'sequence': '<s>The man went skiing with his daughter not knowing that he '
              'would have an accident.</s>',
  'token': 1354,
  'token_str': 'Ġdaughter'},
 {'score': 0.06778453290462494,
  'sequence': '<s>The man went skiing with his wife not knowing that he would '
              'have an accident.</s>',
  'token': 1141,
  'token_str': 'Ġwife'},
 {'score': 0.06104987487196922,
  'sequence': '<s>The man went skiing with his friends not knowing that he '
              'would have an accident.</s>',
  'token': 964,
  'token_str': 'Ġfriends'},
 {'score': 0.0447111539542675,
  'sequence': '<s>The man went skiing with his son not knowing that he would '
              'have an accident.</s>',
  'token': 979,
  'token_str': 'Ġson'}]


# Nächstes Wort erraten
sog. Casual language modeling

In [19]:
from transformers import AutoModelWithLMHead, AutoTokenizer, top_k_top_p_filtering
import torch
from torch.nn import functional as F
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelWithLMHead.from_pretrained("gpt2", return_dict=True)
sequence = f"Its about time we"
input_ids = tokenizer.encode(sequence, return_tensors="pt")
# get logits of last hidden state
next_token_logits = model(input_ids).logits[:, -1, :]
# filter
filtered_next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=50, top_p=1.0)
# sample
probs = F.softmax(filtered_next_token_logits, dim=-1)
next_token = torch.multinomial(probs, num_samples=1)
generated = torch.cat([input_ids, next_token], dim=-1)
resulting_string = tokenizer.decode(generated.tolist()[0])

In [20]:
print(resulting_string)

Its about time we all


# Ganzen text generieren

In [22]:
from transformers import pipeline

text_generator = pipeline("text-generation")
print(text_generator("In the age of data journalism people demand that", max_length=50, do_sample=False))

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "In the age of data journalism people demand that the government take action on their behalf. This has led to a number of stories about the government's handling of the data.\n\nThe government has been accused of using the data to justify its own policies"}]


# Text automatisch zusammenfassen

In [23]:
from transformers import pipeline
summarizer = pipeline("summarization")
ARTICLE = """ David Graeber, anthropologist and anarchist author of bestselling books on bureaucracy and economics including Bullshit Jobs: A Theory and Debt: The First 5,000 Years, has died aged 59.

On Thursday Graeber’s wife, the artist and writer Nika Dubrovsky, announced on Twitter that Graeber had died in hospital in Venice the previous day. The cause of death is not yet known.

Renowned for his biting and incisive writing about bureaucracy, politics and capitalism, Graeber was a leading figure in the Occupy Wall Street movement and professor of anthropology at the London School of Economics (LSE) at the time of his death. His final book, The Dawn of Everything: a New History of Humanity, written with David Wengrow, will be published in autumn 2021.
Caring too much. That's the curse of the working classes

The historian Rutger Bregman called Graeber “one of the greatest thinkers of our time and a phenomenal writer”, while the Guardian columnist Owen Jones called him “an intellectual giant, full of humanity, someone whose work inspired and encouraged and educated so many”. The Labour MP John McDonnell wrote: “I counted David as a much valued friend and ally. His iconoclastic research and writing opened us all up to fresh thinking and such innovative approaches to political activism. We will all miss him hugely.”

Tom Penn, Graeber’s editor at Penguin Random House, said the publishing house was “devastated” and called Graeber “a true radical, a pioneer in everything that he did”.

“David’s inspirational work has changed and shaped the way people understand the world. In his books, his constant, questing curiosity, his wry, sharp-eyed provoking of received nostrums shine through. So too, above all, does his unique ability to imagine a better world, borne out of his own deep and abiding humanity,” Penn said. “We are deeply honoured to be his publisher, and we will all miss him: his kindness, his warmth, his wisdom, his friendship. His loss is incalculable, but his legacy is immense. His work and his spirit will live on.”

Born in New York in 1961 to two politically active parents – his father fought in the Spanish civil war with the International Brigades, while his mother was a member of the international Ladies’ Garment Workers’ Union – Graeber first attracted academic attention for his teenage hobby of translating Mayan hieroglyphs. After studying anthropology at the State University of New York at Purchase and the University of Chicago, he won a prestigious Fulbright fellowship and spent two years doing anthropological fieldwork in Madagascar.
David Graeber interview: ‘So many people spend their working lives doing jobs they think are unnecessary’
Read more

In 2005, Yale decided against renewing his contract a year before he would have secured tenure. Graeber suspected it was because of his politics; when more than 4,500 colleagues and students signed petitions supporting him, Yale instead offered him a year’s paid sabbatical, which he accepted and moved to the UK to work at Goldsmiths before joining LSE. “I guess I had two strikes against me,” he told the Guardian in 2015. “One, I seemed to be enjoying my work too much. Plus I’m from the wrong class: I come from a working-class background.”

His 2011 book Debt: The First 5,000 Years, made him famous. In it, Graeber explored the violence that lies behind all social relations based on money, and called for a wiping out of sovereign and consumer debts. While it divided critics, it attracted strong sales and praise from everyone from Thomas Piketty to Russell Brand.

Graeber followed it in 2013 with The Democracy Project: A History, a Crisis, a Movement, about his work with Occupy Wall Street, then The Utopia of Rules: On Technology, Stupidity and the Secret Joys of Bureaucracy in 2015, which was inspired by his struggle to settle his mother’s affairs before she died. A 2013 article, On the Phenomenon of Bullshit Jobs, led to Bullshit Jobs: A Theory, his 2018 book in which he argued that most white-collar jobs were meaningless and that technological advances had led to people working more, not less."""

In [24]:
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))

[{'summary_text': ' David Graeber, an anthropologist and anarchist, has died aged 59 . He was a leading figure in the Occupy Wall Street movement and professor of anthropology at LSE . His final book, The Dawn of Everything: a New History of Humanity, will be published in 2021 .'}]


# Summary

![alt text](huggin.jpeg "How I felt!")

# Und jetzt sogar noch in Deutsch!

In [25]:
ger_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [26]:
ARTICLE = """Wenn er mit seinem Gilet daherkam und seiner Ledertasche, ein bisschen die Beine schleifen ließ, den Kopf ein wenig nach vorne gebeugt und dabei immer dieses Lachen im Gesicht, da hatte er etwas von einem älter gewordenen Jungen, der viel jünger wirkte, als er war. Und jetzt diese Nachricht: David Graeber ist gestorben, in Venedig. Er wurde 59 Jahre alt.      

Es gibt ja nicht viele Linksradikale, die den Linksradikalismus nicht zur Clownerie verkommen lassen, sondern ernst meinen, und die zugleich zu globalen Superstars und Bestsellerautoren werden. "Anarchist" nannte er sich – oder wurde er genannt –, aber ob er wirklich einer war, das kann man diskutieren. Er war einfach der Meinung, dass Menschen ein solidarisches Miteinander pflegen und aufeinander achtgeben würden, wenn sie nicht in repressiven Strukturen eingepfercht wären, und er war überzeugt, "dass Macht korrumpiert". Anderen linken Strömungen oder gar Parteien fühlte er sich nicht richtig zugehörig, so war er vielleicht eher ein Anarchist mangels besserer Alternative.

Graeber besaß noch etwas von dem Habitus früherer Revolutionäre und Intellektueller, die wichtigtuerische Aufgeblasenheit mancher akademischer Linker war nicht seine Sache, er war da viel bescheidener. Vielleicht hat das auch mit seiner Herkunft zu tun. Graeber wuchs in einer linken, jüdischen Arbeiterklassenfamilie auf, sein Vater kämpfte im Spanischen Bürgerkrieg.

Weltberühmt und zu einer Figur der internationalen Linken machte den US-amerikanischen Anthropologen, der seit Jahren in London lehrte, die Finanzkrise vor zehn Jahren.

Sein Buch Schulden. Die ersten 5.000 Jahre war ein Ereignis, natürlich auch deswegen, weil zockende Banken, weil Kredite, Budgetdefizite von Staaten das Thema der Stunde waren. Graeber zerlegte ein paar Mythen und blickte auf scheinbare Selbstverständlichkeiten mit einem neuen scharfen Blick. Sozialverhältnisse, die von Geld bestimmt werden, produzierten Gewalt, Entmenschlichung, Sklaverei, schrieb er. Zahlungsverhältnisse etablierten hierarchische Verhältnisse von Macht und Ohnmacht. Wo alle verschuldet seien, rennen viele nur mehr um das Überleben.

Ursprungsmythen, wie sie die Ökonomie so liebt, wie das Märchen, dass frühere Gesellschaften einfach Gebrauchswerte tauschten, wischte Graeber vom Tisch: Solche Gesellschaften gab es nie. Immer schon nutzten Menschen Äquivalente zur Vereinfachung des Tausches. Geld aber, in Form von Banknoten und Münzen, war bis in die frühe Neuzeit sehr selten. Man brauchte nicht viel davon, wenn alle anschreiben ließen und höchstens zweimal im Jahr abrechneten.

"""

In [27]:
print(ger_summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))

[{'summary_text': 'David Graeber ist gestorben, in Venedig. "Anarchist" nannte er sich – oder wurde er genannt –. Er war einfach der Meinung, dass Menschen ein solidarisches Miteinander pflegen und aufeinander achtgeben würden.'}]
